https://public.opendatasoft.com/explore/dataset/geonames-all-cities-with-a-population-1000/export/?disjunctive.cou_name_en&sort=name&location=7,51.62484,6.58081&basemap=jawg.streets


In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
pop = pd.read_csv("data/processed/nl3.csv", sep=',',index_col=[0])
dash = pd.read_csv("app/df_coo_pararius.csv")
zio = pd.read_csv("data/processed/zipcodeZ.csv", sep=',',index_col=[0])

In [3]:
searchfor = dash['Plaats'].unique()
pop['Alternate Names'] = pop['Alternate Names'].fillna('None')
for i in searchfor:
    pop.loc[pop['Alternate Names'].str.contains(i),'city'] = i
pop['city'] = pop['city'].fillna(pop['city'])

<ipython-input-3-8065dc9aa161>:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  pop.loc[pop['Alternate Names'].str.contains(i),'city'] = i


In [4]:
ast  =  list(set(searchfor) - set(pop['city']))

In [5]:
test = dash[dash['Plaats'].isin(ast)]
aaa = pd.merge(test[['Plaats','price']],zio[['Plaats','Provincie','longitude','latitude']], left_on='Plaats', right_on='Plaats',how='left')
ccc = aaa.groupby(['Plaats','Provincie'])[['longitude','latitude']].mean().reset_index()

In [6]:
def haversine(lat1, lon1, lat2, lon2):
    earth_radius=6371
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    return earth_radius * 2 * np.arcsin(np.sqrt(a))

ccc['latitude1'] = 51.907183
ccc['longitude1'] = 4.472815
ccc['distance'] = haversine(ccc['latitude'], ccc['longitude'], ccc['latitude1'], ccc['longitude1'])
ccc = ccc.rename(columns={'Plaats':'city','Provincie':'Province'})

In [7]:
finaleira = pd.concat([pop,ccc],0).reset_index(drop=True)

<ipython-input-7-cc23d04fda52>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  finaleira = pd.concat([pop,ccc],0).reset_index(drop=True)


In [8]:
import numpy as np
def haversine(lat1, lon1, lat2, lon2):
    earth_radius=6371
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    return earth_radius * 2 * np.arcsin(np.sqrt(a))

finaleira['latitude1'] = 51.907183
finaleira['longitude1'] = 4.472815
finaleira['distance'] = haversine(finaleira['latitude'], finaleira['longitude'], finaleira['latitude1'], finaleira['longitude1'])
finaleira = finaleira.sort_values('distance')

In [9]:
finaleira['cost'] = finaleira['cost'].interpolate().fillna(finaleira['cost'].mean())

In [10]:
finaleira.to_csv("data/processed/nl3.csv")

In [11]:
average_rent = dash.groupby(['Plaats','Provincie'])[['price']].mean().reset_index().rename(columns={'price':'avg_rent'})
fim = pd.merge(finaleira,average_rent,left_on=['city','Province'],right_on=['Plaats','Provincie'],how='left')
fim['avg_costs'] = fim['avg_rent'] + fim['cost']

In [12]:
fim['latitude' ] = round(fim['latitude'].astype(float),5)
fim['longitude'] = round(fim['longitude'].astype(float),5)
fim['distance'] = round(fim['distance'],2)
fim['cost'] = round(fim['cost'],2)
fim['avg_rent'] = round(fim['avg_rent'],2)
fim['avg_costs'] = round(fim['avg_costs'],2)

In [13]:
final = fim[[
    'Alternate Names',
    'city',
    'Province',
    'Population',
    'distance',
    'cost',
    'avg_rent',
    'avg_costs',
    'latitude',
    'longitude',
]].fillna('None')
final

,Alternate Names,city,Province,Population,distance,cost,avg_rent,avg_costs,latitude,longitude
0,"RTM,Ratehrdam,Roterdam,Roterdama,Roterdamas,Ro...",Rotterdam,Zuid-Holland,598199.0,1.76,857.05,1499.02,2356.07,51.92250,4.47917
1,"Schidamas,Schiedam,Skhidam,Skiedam,Sxidam,askh...",Schiedam,Zuid-Holland,75438.0,5.91,857.05,1271.72,2128.77,51.91917,4.38889
2,"Barendrekht,Barendrext,ba lun de lei he te,bar...",Barendrecht,Zuid-Holland,38672.0,7.04,847.63,1747.5,2595.13,51.85667,4.53472
3,"Capelle,Capelle a/d IJssel,Capelle a/d Yssel,C...",Capelle aan den IJssel,Zuid-Holland,65255.0,7.60,842.92,1101.24,1944.16,51.92917,4.57778
4,Hoogvliet Rotterdam,Hoogvliet Rotterdam,Zuid-Holland,34950.0,9.01,835.85,1143.75,1979.6,51.86333,4.36250
...,...,...,...,...,...,...,...,...,...,...
435,None,Farmsum,Groningen,2185.0,228.40,877.61,0.0,877.61,53.32167,6.92639
436,None,Roodeschool,Groningen,None,228.48,877.61,1050.0,1927.61,53.41983,6.76638
437,"Delfzejl,Delfzijl,Delfzyl,Делфзейл",Delfzijl,Groningen,28649.0,228.62,877.61,680.6,1558.21,53.33000,6.91806
438,Spijk Gn,Spijk,Groningen,1220.0,229.40,877.61,1500.0,2377.61,53.39000,6.83750


In [14]:
# writer = pd.ExcelWriter('output.xlsx')
# final[final['distance']<=23].to_excel(writer, 'Sugestion')
# final.to_excel(writer, 'All')
# writer.save()

In [15]:
# final[final['city'].isin(searchfor)].reset_index(drop=True).to_csv('app/costs.csv')

In [16]:
searchfor = ["'s-Hertogenbosch",
 'Alkmaar',
 'Almere',
 'Amersfoort',
 'Amstelveen',
 'Amsterdam',
 'Apeldoorn',
 'Arnhem',
 'Breda',
 'Bussum',
 'Delft',
 'Dordrecht',
 'Eindhoven',
 'Gouda',
 'Haarlem',
 'Hilversum',
 'Hoofddorp',
 'Huizen',
 'Leiden',
 'Oegstgeest',
 'Rijswijk',
 'Rotterdam',
 'Schiedam',
 'Den Haag',
 'Utrecht',
 'Voorburg',
 'Zaandam',
 'Zeist',
 'Zoetermeer']

In [17]:
final[final['city'].isin(searchfor)].reset_index(drop=True).to_csv('app/costs.csv')

In [18]:
asdasdasdasdas

NameError: name 'asdasdasdasdas' is not defined

In [ ]:
finaleira['price'] = round(finaleira['price'],2)



In [ ]:
finaleira = finaleira[finaleira['city'].isin(set(dash['City']))].drop_duplicates(['Province','city'])

finaleira[[
    'city',
    'Province',
    'price',
    'distance',
    'Population',
    'Alternate Names',
    'latitude',
    'longitude',
]].rename(columns={'price':'costs'}).to_csv('app/costs.csv',index=False)

In [ ]:
asdasdasds

In [ ]:
def impute(df2, col_name):
    while df2[col_name].isna().any().any():
        first_na = df2[col_name].isna().diff() & df2[col_name].isna()
        imputed = df2.rolling(3, min_periods=1).mean().shift()[col_name]
        df2.loc[first_na, col_name] = imputed
        
impute(finaleira, 'price')

In [ ]:
# finaleira['price']= finaleira['price'].fillna(pd.rolling_mean(finaleira['price'], 6, min_periods=1))
pd.Series(finaleira['price']).rolling(window=12).mean()

In [ ]:
(finaleira.groupby(finaleira['price'].notnull().cumsum(),as_index=False)['price'].transform(lambda x: x.iloc[0]/x.size))

In [ ]:
all_cities = pd.read_csv("data/processed/nl.csv")
zipcode = pd.read_csv("data/processed/zipcodeZ.csv",index_col=['Unnamed: 0'])

In [ ]:
all_cities['city_']  = all_cities['city']   + "_" + all_cities['Provincie']
zipcode['validador1'] = zipcode['Plaats']   + "_" + zipcode['Provincie']
zipcode['validador2'] = zipcode['Gemeente'] + "_" + zipcode['Provincie']

In [ ]:
City       = all_cities['city_'].unique()
validador1 = zipcode['validador1'].unique()
validador2 = zipcode['validador2'].unique()

In [ ]:
listin = (list(validador1) + list(validador2))
minha_lista = set(listin) - set(City)
len(minha_lista)

In [ ]:
lat=[]
lon=[]
city_=[]
provincie_=[]
error=[]
i=0

for place in minha_lista:
    x = place.split("_")
    city=x[0]
    Provincie=x[1]
    place = city+" "+Provincie+" "+'Netherlands'
    
    try:
        url = "https://nominatim.openstreetmap.org/search?q="+place+"&format=geojson"
        r = requests.get(url)
        lat.append(r.json()['features'][0]['geometry']['coordinates'][1])
        lon.append(r.json()['features'][0]['geometry']['coordinates'][0])
        city_.append(city)
        provincie_.append(Provincie)
        print(place)
        
    except:
        print('##########', place)
        error.append(place)
        pass
    df = pd.DataFrame({'latitude':lat,'longitude':lon,'city':city_,'provincie':provincie_})
    df.to_csv(f"data/processed/temp/{i}.csv")
    i=+1

In [ ]:
"https://nominatim.openstreetmap.org/search?q="+place+"&format=geojson"

In [ ]:
new_cities = pd.DataFrame({'latitude':lat,'longitude':lon,'city':city_,'provincie':provincie_})

In [ ]:
all_cities_processed = pd.concat([all_cities,new_cities],0)
all_cities_processed['Provincie'] = all_cities_processed['Provincie'].fillna(all_cities_processed['provincie'])

In [ ]:
all_cities_processed['latitude'] = round(all_cities_processed['latitude'],5)
all_cities_processed['longitude'] = round(all_cities_processed['longitude'],5)

In [ ]:
all_cities_processed = all_cities_processed[~all_cities_processed['Provincie'].isna()]
all_cities_processed = all_cities_processed[['city', 'latitude', 'longitude','Provincie', 'population']]
all_cities_processed = all_cities_processed.drop_duplicates(['city', 'Provincie'])
all_cities_processed = all_cities_processed.reset_index(drop=True)
all_cities_processed.to_csv("data/processed/nl.csv",index=False)

In [ ]:
all_cities_processed[all_cities_processed['city'].str.contains('Den Haag|The Hague')]

In [ ]:
all_cities_processed

In [ ]:
# ids = all_cities_processed["city"]
# all_cities_processed[ids.isin(ids[ids.duplicated()])].sort_values("city").head(50)

In [ ]:
pop = pd.read_csv("data/processed/csvData.csv")
pop1 = pd.merge(all_cities_processed,pop,left_on='city',right_on='name',how='left').drop(columns=['name'])
pop1['popz'] = pop1['2021'      ].fillna(pop1['population'])
pop1['popz'] = pop1['population'].fillna(pop1['2021'])
pop1[~pop1['popz'].isna()]

In [ ]:
#pop1[pop1['city'].str.contains('s-')] #Den Haag|The Hague|

In [ ]:
all_cities_processed['latitude__'] = all_cities_processed['latitude'].astype(str) + "_" + all_cities_processed['longitude'].astype(str)

In [ ]:
# ids = all_cities_processed["latitude__"]
# all_cities_processed[ids.isin(ids[ids.duplicated()])].sort_values("latitude__").head(10)

In [ ]:
dasdasdasdasdasdasdasdasdasdas

In [ ]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import requests

In [ ]:
URL = 'https://www.numbeo.com/cost-of-living/country_result.jsp?country=Netherlands'
response = get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
city_form = soup.find('form', {'class': 'standard_margin'})
city_form = [values['value'] for values in city_form('option')]
final = pd.DataFrame(city_form)

writer = pd.ExcelWriter('output.xlsx')
final.to_excel(writer, 'All')
writer.save()

In [ ]:
for city in city_form[:]:
    URL = 'https://www.numbeo.com/cost-of-living/city-estimator/in/'
    IRL = '?Recalculated=Submit+to+Recalculate&displayCurrency=EUR&members=1&restaurants_percentage=10.0&inexpensive_restaurants_percentage=50.0&drinking_coffee_outside=100.0&going_out_monthly=4.2&smoking_packs_per_day=0.0&alcoholic_drinks=25.0&type_of_food=0&driving_car=0.0&taxi_consumption=0.0&paying_for_public_transport=Monthly%2C+All+Members&sport_memberships=100.0&vacation=0.0&clothing_and_shoes=50.0&rent=none&kindergarten_count=0&private_schools_count=0'
    benefit=[]
    price=[]
    city = city.replace(' ','-')
    city = city.replace('(','')
    city = city.replace(')','')
    try:
        link = URL+city+IRL
        response = get(URL+city+IRL)
        soup = BeautifulSoup(response.text, 'html.parser')
        if len(soup.find_all('nav',{'class':'breadcrumb'}))==0:
            link=URL+city+'-Netherlands'+IRL
            response = get(link)
            soup = BeautifulSoup(response.text, 'html.parser')
        else:
            pass
        benefit = (soup.find_all('th',{'class':'tr_highlighted_menu'})[-1].get_text())
        price = (soup.find_all('th',{'class':'th_no_highlight_a_right'})[-1].get_text())
        meu_01.append({'benefit':benefit,'price':price,'city':city,'url':link})
    except:
        pass

In [ ]:
df = pd.DataFrame(meu_01)
df = df[df['price']!=' ?'].reset_index(drop=True)
df['price'] = df['price'].str.replace('?','0',regex=True)
df['price'] = df['price'].str.replace('\D','',regex=True)
df['price'] = (df['price'].astype(float))/100

In [ ]:
df.loc[df['city']=='The-Hague-Den-Haag','city'] = 'The Hague'
df.loc[df['city']=='Halle','city'] = 'Hall'
df.loc[df['city'].str.contains('Hertogenbosch'),'city'] = "’s-Hertogenbosch"

In [ ]:
df

In [ ]:
df['address'] = address
df['latitude'] = latitude
df['longitude'] = longitude

df['latitude'] = round(df['latitude'],7)
df['longitude'] = round(df['longitude'],7)

In [ ]:
df['benefit'] = 'costs'
df['link'] = "<a target='_blank' href=" + (df['url']).astype(str) + ">" + 'Source' + "</a>"
df = df.drop_duplicates().sort_values('price', ascending=False).reset_index(drop=True)

In [ ]:
import numpy as np
def haversine(lat1, lon1, lat2, lon2):
    earth_radius=6371
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    return earth_radius * 2 * np.arcsin(np.sqrt(a))

df['latitude1'] = 51.907183
df['longitude1'] = 4.472815
df['distance'] = haversine(df['latitude'], df['longitude'], df['latitude1'],df['longitude1'])
df['distance'] = round(df['distance'],2)

In [ ]:
df

In [ ]:
ksjdhakjsdhkjashdkjashdkjahskdjhaskjdhaskjds
ksjdhakjsdhkjashdkjashdkjahskdjhaskjdhaskjds
ksjdhakjsdhkjashdkjashdkjahskdjhaskjdhaskjds

In [ ]:
df.reset_index(drop=True).to_csv('../../app/costs.csv')

In [ ]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [ ]:
def  minha(URL,city):
    response = get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    split1 = soup.find_all('tr',{'class':'tr_standard'})
    benefit = []
    col_factor = []
    price = []
    city_form=[]
    for i in range(len(split1)):
        try:
            benefit_ = split1[i].find('td').find(text=True)
        except:
            benefit_ = None
        try:
            col_factor_ = split1[i].find('span',{'class':'estimator_multiplier'}).find(text=True)
        except:
            col_factor_ = None
        try:
            price_ = split1[i].find('span',{'class':'price_in_units_for_analysis'}).find(text=True)
        except:
            price_ = None

        benefit.append(benefit_)
        col_factor.append(col_factor_)
        price.append(price_)
        city_form.append(city)
        
    return [benefit,col_factor,city_form,price]

In [ ]:
URL = 'https://www.numbeo.com/cost-of-living/country_result.jsp?country=Netherlands'
response = get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
city_form = soup.find('form', {'class': 'standard_margin'})
city_form = [values['value'] for values in city_form('option')]
meu_01=[]

In [ ]:
for city in city_form[:]:
    URL = 'https://www.numbeo.com/cost-of-living/city-estimator/in/'+city
    my_dict_ = minha(URL,city)
    meu_01.append({'benefit':my_dict_[0],'col_factor':my_dict_[1],'city':my_dict_[2],'price':my_dict_[3]})

In [ ]:
df = pd.DataFrame({})
for i in range(len(meu_01)):
    df = pd.concat([df, pd.DataFrame(meu_01[i])],0)

In [ ]:
df['price'] = df['price'].str.replace('?','0',regex=True)
df['price']= df['price'].str.replace('\D','',regex=True)
df['price'] = (df['price'].astype(float))/100

In [ ]:
df['col_factor']= df['col_factor'].str.replace('x','',regex=True)
df['col_factor'] = (df['col_factor'].astype(float))/2

---

In [ ]:
# Import Libs
from folium.plugins import FastMarkerCluster
import io
import base64
import folium
import pandas as pd
import streamlit as st
from st_aggrid import AgGrid
import matplotlib.cm as cm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import folium.plugins
import branca
import branca.colormap as cm
import folium.plugins
import branca
import branca.colormap as cm

In [ ]:
final = pd.read_csv('../../app/costs.csv', index_col=[0])
final_ = final[['price', 'city', 'url']].to_html(escape=False)
lats = final['latitude'].tolist()
lons = final['longitude'].tolist()
city = final['city'].tolist()
price = final['price'].tolist()
map = folium.Map(location=[52.2129919, 5.2793703],zoom_start=7, tiles="cartodbdark_matter")
feature_group = folium.FeatureGroup("Locations")
for lats, lons, price, city in zip(lats, lons, price, city):
    feature_group.add_child(
        folium.CircleMarker(
            location=[lats, lons],
            popup=f"{city}, €{price}",
            radius=4,
            fill=True,
        )
    )
map.add_child(feature_group)

In [ ]:
fname = 'test_colorbar.html'


base_lat = 50.682294344
base_lon = 10.939628989
f_map = folium.Map(location=[base_lat,base_lon], tiles="OpenStreetMap", zoom_start=17)

lat_ = [50.6823, 50.6822, 50.6821, 50.6820, 50.6819]
lon_ = [10.9396291, 10.9396279, 10.9396269, 10.9396265, 10.9396261]
acc_ = [11.0,44.5,149.9,319.1,540.0,752.6]

colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
                             index=[0, 25, 62.5, 156.25, 390.6, 1000], vmin=0, vmax=1000,
                             caption='Total Standard deviation at the point[mm]')

fg = folium.FeatureGroup(name=fname.split('.')[0])    

for pt in range(len(lat_)):

    color = colormap(acc_[pt])
    fg.add_child(folium.CircleMarker(location=[lat_[pt],lon_[pt]],
                                     radius=6,
                                     fill=True,
                                     color=color,
                                     fill_color=color))

f_map.add_child(fg)
f_map.add_child(colormap)
f_map

In [ ]:
final = pd.read_csv('../../app/costs.csv', index_col=[0])

map = folium.Map(location=[52.2129919, 5.2793703],zoom_start=7, tiles="cartodbdark_matter")
feature_group = folium.FeatureGroup("Locations")

lats = final['latitude'].tolist()
lons = final['longitude'].tolist()
city = final['city'].tolist()
price = final['price'].tolist()
acc_ = final['price'].unique().tolist()

for lats, lons, price, city in zip(lats, lons, price, city):
    color = colormap(acc_.index(price))
    feature_group.add_child(
        folium.CircleMarker(
            location=[lats, lons],
            radius=6,
            fill=True,
            color=color,
            fill_color=color))
map.add_child(feature_group)

In [ ]:
for lat, lon, traffic_q, traffic, bike, city in zip(df['latitude'], df['longitude'], df['traffic_index_quartile'], df['traffic_index'], df['bike_score'], df['city']):
    folium.CircleMarker(
        [lat, lon],
        radius=.15*bike,
        popup = ('City: ' + str(city).capitalize() + '<br>'
                 'Bike score: ' + str(bike) + '<br>'
                 'Traffic level: ' + str(traffic) +'%'
                ),
        color='b',
        key_on = traffic_q,
        threshold_scale=[0,1,2,3],
        fill_color=colordict[traffic_q],
        fill=True,
        fill_opacity=0.7
        ).add_to(traffic_map)
traffic_map

In [ ]:
import seaborn as sns
print(sns.palplot(sns.color_palette("coolwarm", 33)))
print(sns.palplot(sns.color_palette("flare", 33)))
print(sns.palplot(sns.color_palette("viridis", 33)))
print(sns.palplot(sns.color_palette("Spectral", 33)))




In [ ]:
up = sns.palplot(sns.color_palette("Spectral", 33))

In [ ]:
rgb = [(0.15, 0.25, 0.35), (0.35, 0.55, 0.51), (0.5, 0.95, 0.34)]
= sns.palplot(sns.blend_palette(rgb, n_colors=10))


In [ ]:
sns.deep.values

In [ ]:
pal = sns.color_palette("magma")
print(pal)
sns.palplot(pal)

In [ ]:
color_pallete = (sns.color_palette("Spectral", 33))
color_pallete = color_pallete.as_hex()
c = color_pallete[1]
c